<a href="https://colab.research.google.com/github/xuxinyue18-dot/deepseekAPI/blob/main/deepseek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 使用说明
1. 首次运行前，请确保已经安装 `openai`、`requests`、`beautifulsoup4` 等依赖。
2. 请在运行前将 `DEEPSEEK_API_KEY` 设置为有效的 DeepSeek API 密钥。若需启用联网搜索功能，请额外配置 `SERPER_API_KEY`。
3. 可以通过环境变量或其他安全方式提供密钥，避免将密钥写入代码。
4. 运行最后一个单元格后，即可通过命令行与 DeepSeek 进行对话。输入 `退出` 结束对话，输入 `清除` 清除上下文。


In [ ]:
# 如果在全新环境中运行，请先安装所需依赖。
# 在 Jupyter/Colab 中可以取消注释下一行命令：
# %pip install openai requests beautifulsoup4


In [ ]:
import os
import sys
import time
from typing import Any, Dict, List, Union

import requests
from bs4 import BeautifulSoup
from openai import OpenAI

BASE_URL = "https://api.deepseek.com"
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY", "YOUR_DEEPSEEK_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")


def _require_api_key(value: str, env_name: str) -> str:
    """确保已经提供了对应的 API Key。"""
    if not value or value.startswith("YOUR_"):
        raise ValueError(
            f"请先设置 {env_name} 环境变量，或在代码中替换占位符为真实密钥。"
        )
    return value


client = OpenAI(
    api_key=_require_api_key(DEEPSEEK_API_KEY, "DEEPSEEK_API_KEY"),
    base_url=BASE_URL,
)


In [ ]:
def print_streaming(text: str, delay: float = 0.02) -> None:
    """以打字机效果打印文本。"""
    for char in text:
        sys.stdout.write(char)
        sys.stdout.flush()
        time.sleep(delay)
    sys.stdout.write("\n")
    sys.stdout.flush()


def search_web(query: str, num_results: int = 3) -> Union[Dict[str, Any], str]:
    """使用 Serper API 进行搜索。"""
    query = query.strip()
    if not query:
        return "搜索关键词不能为空。"
    if not SERPER_API_KEY:
        return "搜索功能未启用：请先配置 SERPER_API_KEY。"

    payload = {"q": query, "num": num_results}
    headers = {
        "X-API-KEY": SERPER_API_KEY,
        "Content-Type": "application/json",
    }

    try:
        response = requests.post("https://google.serper.dev/search", json=payload, headers=headers, timeout=15)
        response.raise_for_status()
        data = response.json()
        if not data.get("organic"):
            return "搜索未返回有效结果。"
        return data
    except requests.RequestException as exc:
        return f"搜索请求失败：{exc}"
    except ValueError:
        return "搜索响应解析失败。"


def get_webpage_content(url: str) -> str:
    """抓取网页并提取纯文本内容。"""
    try:
        headers = {
            "User-Agent": (
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                "(KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
            )
        }
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        for script in soup(["script", "style"]):
            script.decompose()
        text = soup.get_text(separator="\n", strip=True)
        lines = [line for line in (segment.strip() for segment in text.split("\n")) if line]
        return "\n".join(lines)[:1000]
    except requests.RequestException as exc:
        return f"获取网页内容出错：{exc}"


def optimize_prompt(prompt: str) -> str:
    """调用 DeepSeek 对提示词进行润色。"""
    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": "请优化以下提示词，使其更加清晰和专业。"},
                {"role": "user", "content": prompt},
            ],
            stream=False,
        )
        content = response.choices[0].message.content.strip()
        return content or prompt
    except Exception as exc:
        print_streaming(f"提示词优化失败，将使用原始问题：{exc}")
        return prompt


def get_ai_streaming_response(messages: List[Dict[str, str]]) -> str:
    """以流式方式获取 DeepSeek 回复。"""
    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=messages,
            stream=True,
        )
    except Exception as exc:
        error_message = f"API 调用失败：{exc}"
        print_streaming(error_message)
        return error_message

    full_reply = []
    sys.stdout.write("助手: ")
    sys.stdout.flush()

    for chunk in response:
        delta = chunk.choices[0].delta.content
        if delta:
            full_reply.append(delta)
            sys.stdout.write(delta)
            sys.stdout.flush()
            time.sleep(0.01)

    sys.stdout.write("\n")
    sys.stdout.flush()
    return "".join(full_reply).strip()


SYSTEM_PROMPT = {
    "role": "system",
    "content": (
        "You are a helpful assistant. When users explicitly ask to 搜索, summarise "
        "web results before answering. Otherwise, rely on your built-in knowledge."
    ),
}


def run_cli_assistant() -> None:
    """通过命令行与 DeepSeek 对话。"""
    print_streaming("欢迎使用 DeepSeek 智能助手！")
    print("功能说明：")
    print("1. 直接输入问题进行对话")
    print("2. 输入'搜索：关键词'可触发联网搜索")
    print("3. 输入'清除'清除对话历史")
    print("4. 输入'退出'结束对话\n")

    messages: List[Dict[str, str]] = [SYSTEM_PROMPT.copy()]

    while True:
        user_input = input("用户: ").strip()

        if user_input.lower() in {"退出", "exit"}:
            print_streaming("对话已结束，再见！")
            break

        if user_input.lower() in {"清除", "clear"}:
            messages = [SYSTEM_PROMPT.copy()]
            print_streaming("历史记录已清除！")
            continue

        if user_input.startswith(("搜索:", "搜索：")):
            normalized = user_input.replace("搜索：", "搜索:", 1)
            if ":" in normalized:
                search_query = normalized.split(":", 1)[1].strip()
            else:
                search_query = user_input[len("搜索"):].strip(" ：:")
            if not search_query:
                print_streaming("请输入搜索关键词！")
                continue
            print_streaming(f"正在搜索：{search_query}")
            search_results = search_web(search_query)

            if isinstance(search_results, str):
                print_streaming(f"搜索失败，将直接使用 AI 回答。（{search_results}）")
                user_input = search_query
            else:
                combined_info: List[str] = []
                for result in search_results.get("organic", [])[:2]:
                    title = result.get("title", "")
                    link = result.get("link", "")
                    snippet = result.get("snippet", "")
                    content = get_webpage_content(link)
                    combined_info.append(
                        f"标题: {title}\n链接: {link}\n摘要: {snippet}\n内容: {content}\n"
                    )

                if combined_info:
                    user_input = (
                        f"请基于以下搜索结果回答关于 '{search_query}' 的问题：\n\n"
                        + "\n---\n".join(combined_info)
                    )
                else:
                    user_input = f"搜索 '{search_query}' 没有找到足够的信息，请结合已有知识回答。"

        print_streaming("正在优化提示词...")
        optimized_prompt = optimize_prompt(user_input)
        print_streaming(f"优化后的提示词：{optimized_prompt}")

        messages.append({"role": "user", "content": optimized_prompt})
        assistant_reply = get_ai_streaming_response(messages)
        messages.append({"role": "assistant", "content": assistant_reply})
        print()


run_cli_assistant()
